In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime
import os

import geopandas as gpd
import keras

import gee
import utils

In [4]:
region_name = 'amazon_1'

tile_size = 576 # this is the around the max size that GEE exports allow with 12-band imagery
tile_padding = 24

start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 12, 1)
clear_threshold = 0.6

pred_threshold = 0.5

In [5]:
# load a list of models
model_versions = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i']
model_names = [f'48px_v3.7-{v}_2024-01-17' for v in model_versions]
models = [keras.models.load_model(f'../models/{model_name}.h5') for model_name in model_names]

Metal device set to: Apple M3 Pro

systemMemory: 36.00 GB
maxCacheSize: 13.50 GB



2024-01-17 11:06:29.892947: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-17 11:06:29.893053: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
region = gpd.read_file(f'../data/boundaries/amazon_basin/{region_name}.geojson').geometry[0].__geo_interface__
tiles = utils.create_tiles(region, tile_size, tile_padding)
print(f"Created {len(tiles):,} tiles")

data_pipeline = gee.S2_Data_Extractor(
    tiles, 
    start_date, 
    end_date, 
    clear_threshold, 
    batch_size=500
    )

Created 26,023 tiles


In [9]:
data_pipeline.make_predictions(models, pred_threshold=pred_threshold)

2024-01-17 11:07:13.789814: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-01-17 11:07:13.853041: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-01-17 11:07:14.191268: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-01-17 11:07:14.494443: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-01-17 11:07:14.794482: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-01-17 11:07:15.093052: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-01-17 11:07:15.386223: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114

Error in model.predict for model 3 and tile 576:24:10.0:17:55:-9
{{function_node __wrapped__StridedSlice_device_/job:localhost/replica:0/task:0/device:GPU:0}} Expected begin, end, and strides to be 1D equal size tensors, but got shapes [1], [2], and [1] instead. [Op:StridedSlice] name: strided_slice/
Input shape: (529, 48, 48, 12)
Error in model.predict for model 4 and tile 576:24:10.0:17:56:-5
{{function_node __wrapped__Pack_N_17_device_/job:localhost/replica:0/task:0/device:GPU:0}} Shapes of all inputs must match: values[0].shape = [1] != values[8].shape = [0] [Op:Pack] name: stack
Input shape: (529, 48, 48, 12)


2024-01-17 11:31:41.389568: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at strided_slice_op.cc:111 : INVALID_ARGUMENT: Expected begin, end, and strides to be 1D equal size tensors, but got shapes [1], [2], and [1] instead.


2024-01-17 11:41:39.766052: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at reduction_ops_common.h:147 : INVALID_ARGUMENT: Invalid reduction dimension (32 for input with 1 dimension(s)


Error in model.predict for model 6 and tile 576:24:10.0:18:48:-55
{{function_node __wrapped__All_device_/job:localhost/replica:0/task:0/device:CPU:0}} Invalid reduction dimension (32 for input with 1 dimension(s) [Op:All]
Input shape: (529, 48, 48, 12)
Error in model.predict for model 5 and tile 576:24:10.0:18:44:-57
{{function_node __wrapped__Pack_N_17_device_/job:localhost/replica:0/task:0/device:GPU:0}} Shapes of all inputs must match: values[0].shape = [1] != values[8].shape = [0] [Op:Pack] name: stack
Input shape: (529, 48, 48, 12)


In [20]:
# write the predictions to a file
print(len(preds), 'chips with predictions above', pred_threshold)
# write the predictions to a file
model_version_name = '_'.join(model_name.split('_')[0:2])
# if the outputs directory does not exist, create it
if not os.path.exists(f'../data/outputs/{model_version_name}'):
    os.makedirs(f'../data/outputs/{model_version_name}')
time_period = f"{start_date.month}_{start_date.year}-{end_date.month}_{end_date.year}"
preds.to_file(f"../data/outputs/{model_version_name}/{region_name}_{model_version_name}_{pred_threshold:.2f}_{time_period}.geojson", driver="GeoJSON")

20750 chips with predictions above 0.5
